In [6]:
from openai import OpenAI

from langchain_openai import ChatOpenAI
from langchain_community.document_loaders.csv_loader import CSVLoader

from langchain.chains import RetrievalQA  
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

import json

import pandas as pd
import os

ImportError: cannot import name 'OpenAI' from 'openai' (c:\Users\tjddn\anaconda3\envs\smartchef\lib\site-packages\openai\__init__.py)

In [2]:
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']

chat_model = ChatOpenAI(api_key=OPENAI_API_KEY)
pc = Pinecone(api_key=PINECONE_API_KEY)

AttributeError: module 'openai' has no attribute 'OpenAI'

In [36]:
df = pd.read_csv('./data/receipe_dataset_demo.csv')
df.head(1)

,DishName,CookingMethod,CookingOccasion,MainIngredients,CuisineType,DishDescription,IngredientDetails,ServingSize,DifficultyLevel,CookingTime
0,어묵김말이,튀김,간식,가공식품류,디저트,맛있는 김말이에 쫄깃함을 더한 어묵 김말이예요-,[재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개|...,2인분,초급,60분이내


In [37]:
def process_ingredients(ingredient_str):
    ingredient_str = ingredient_str.replace("[재료] ", "")
    return [f"'{ingredient.strip()}'" for ingredient in ingredient_str.split('|')]

df['Ingredients'] = df['IngredientDetails'].apply(process_ingredients)

In [39]:
df = df[['DishName', 'CookingTime', 'Ingredients']]
df

,DishName,CookingTime,Ingredients
0,어묵김말이,60분이내,"['어묵 2개', '김밥용김 3장', '당면 1움큼', '양파 1/2개', '당근 ..."
1,두부새우전,30분이내,"['두부 1/2모', '당근 1/2개', '고추 2개', '브로콜리 1/4개', '..."
2,알밥,30분이내,"['밥 1+1/2공기', '당근 1/4개', '치자단무지 1/2개', '신김치 1쪽..."
3,현미호두죽,30분이내,"['현미 4컵', '찹쌀 2컵', '호두 50g', '물 1/2컵', '소금 약간']"
4,북어갈비,60분이내,"['북어포 1마리', '찹쌀가루 1C [양념] 간장 2T', '설탕 1T', '물 ..."
5,토마토스파게티,30분이내,"['파스타면 [양념] 토마토 1개', '토마토 페이스트 3T', '양파 1/2개',..."
6,표고버섯탕수,30분이내,"['건표고버섯 9개', '오이 1/2개', '당근 1/2개', '양파 1/2개', ..."
7,갈릭치킨,60분이내,"['닭 1마리 [닭양념] 통마늘 3개', '다진마늘 1T', '화이트와인또는우유',..."
8,양파링튀김,30분이내,"['양파1개', '빵가루', '달걀 노른자1개', '밀가루1C', '물 약간', '..."
9,참치삼각김밥,30분이내,"['밥 2공기', '참치캔 큰거 1개', '양파', '다진김치', '김밥용김 [양념..."


In [53]:
df.iloc[1,2]

["'두부 1/2모'",
 "'당근 1/2개'",
 "'고추 2개'",
 "'브로콜리 1/4개'",
 "'새우 4마리'",
 "'녹말가루'",
 "'계란 1개'"]

In [52]:
index_name = "receipe_ingredients"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=2,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws', 
            region='us-east-1'
        )
    )

PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '9d91e97595e9e02d583379f42f2e72ec', 'Date': 'Thu, 15 Aug 2024 05:02:17 GMT', 'Server': 'Google Frontend', 'Content-Length': '125', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"INVALID_ARGUMENT","message":"Name must consist of lower case alphanumeric characters or '-'"},"status":400}


In [45]:
index = pc.Index(index_name)

index.upsert(
    vectors=[
        {"id": "vec1", "values": [1.0, 1.5]},
        {"id": "vec2", "values": [2.0, 1.0]},
        {"id": "vec3", "values": [0.1, 3.0]},
    ],
    namespace="ns1"
)

index.upsert(
    vectors=[
        {"id": "vec1", "values": [1.0, -2.5]},
        {"id": "vec2", "values": [3.0, -2.0]},
        {"id": "vec3", "values": [0.5, -1.5]},
    ],
    namespace="ns2"
)

upserted_count: 3

In [46]:
print(index.describe_index_stats())

{'dimension': 2,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 3}, 'ns2': {'vector_count': 3}},
 'total_vector_count': 6}


In [47]:
query_results1 = index.query(
    namespace="ns1",
    vector=[1.0, 1.5],
    top_k=3,
    include_values=True
)

print(query_results1)

query_results2 = index.query(
    namespace="ns2",
    vector=[1.0,-2.5],
    top_k=3,
    include_values=True
)

print(query_results2)

{'matches': [{'id': 'vec1',
              'metadata': None,
              'score': 1.0,
              'sparse_values': {'indices': [], 'values': []},
              'values': [1.0, 1.5]},
             {'id': 'vec2',
              'metadata': None,
              'score': 0.86824316,
              'sparse_values': {'indices': [], 'values': []},
              'values': [2.0, 1.0]},
             {'id': 'vec3',
              'metadata': None,
              'score': 0.85006815,
              'sparse_values': {'indices': [], 'values': []},
              'values': [0.1, 3.0]}],
 'namespace': 'ns1',
 'usage': {'read_units': 6}}
{'matches': [{'id': 'vec1',
              'metadata': None,
              'score': 1.0,
              'sparse_values': {'indices': [], 'values': []},
              'values': [1.0, -2.5]},
             {'id': 'vec3',
              'metadata': None,
              'score': 0.9982744,
              'sparse_values': {'indices': [], 'values': []},
              'values': [0.5, 

In [48]:
pc.delete_index(index_name)

In [ ]:
openai.api_key = os.getenv('OPENAI_API_KEY') or 'OPENAI_API_KEY'

embed_model = "text-embedding-ada-002"

In [ ]:
query = "how do I use the LLMChain in LangChain?"

res = openai.Embedding.create(
    input=[query],
    engine=embed_model
)

# retrieve from Pinecone
xq = res['data'][0]['embedding']

# get relevant contexts (including the questions)
res = index.query(vector=xq, top_k=5, include_metadata=True)

In [60]:
from openai import OpenAI
client = OpenAI()

# response = client.embeddings.create(
#     input="Your text string goes here",
#     model="text-embedding-3-small"
# )

# print(response.data[0].embedding)

In [ ]:
openai.api_key = OPENAI_API_KEY
openai.Embedding.create(
  model="text-embedding-ada-002",
  input="The food was delicious and the waiter..."
)

In [ ]:
# 이름, 시간, 요리법, 재료, 추가필요재료